# SARIMAX forecasting on PV

#### Import data

In [45]:
import warnings
warnings.filterwarnings("ignore")
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import pandas as pd
import numpy as np

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import acf, pacf
from sktime.forecasting.arima import ARIMA

import DataRetriever as dr

retriever = dr.DataRetriever()
#year1 = retriever.get_data("All-Subsystems-hour-Year1.pkl")
year2 = retriever.get_data("All-Subsystems-hour-Year2.pkl")
#year2 = year2[0:8000]
year2

,Timestamp,Load_LatentHeatWaterVolume,Load_RefrigeratorTemp,Load_StatusBA1Lights,Load_StatusKitchenLightsA,Load_StatusKitchenLightsB,Load_StatusKitchenLightsC,Load_StatusDRLights,Load_StatusLRLights3,Load_StatusEntryHallLights,...,SHW_GlycolFlowHXCoriolisSHW,SHW_WaterFlowHXCoriolisSHW,SHW_GlycolFlowRateHXCoriolisSHW,SHW_WaterFlowRateHXCoriolisSHW,HVAC_HeatPumpIndoorUnitPower,HVAC_HeatPumpOutdoorUnitPower,HVAC_DehumidifierPower,HVAC_DehumidifierInletAirTemp,HVAC_DehumidifierExitAirTemp,HVAC_DehumidifierAirflow
Timestamp,,,,,,,,,,,,,,,,,,,,,
2015-02-01 00:00:00,2015-02-01 00:00:00,0.060105,4.957915,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,NaN,NaN,NaN,NaN,175.449170,1045.817746,4.498593,72.228102,74.959034,0.0
2015-02-01 01:00:00,2015-02-01 01:00:00,0.131439,4.904702,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,NaN,NaN,NaN,NaN,391.606267,1042.026917,4.442783,71.963450,73.979483,0.0
2015-02-01 02:00:00,2015-02-01 02:00:00,0.194187,5.054887,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,NaN,NaN,NaN,NaN,388.484683,1023.145217,4.444083,71.921017,73.818717,0.0
2015-02-01 03:00:00,2015-02-01 03:00:00,0.258256,4.991525,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,NaN,NaN,NaN,NaN,175.792833,1066.177350,4.467983,71.844683,74.535050,0.0
2015-02-01 04:00:00,2015-02-01 04:00:00,0.322324,5.040868,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,NaN,NaN,NaN,NaN,371.120983,1081.112783,4.466550,71.959983,74.741467,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-01-31 19:00:00,2016-01-31 19:00:00,1.472255,4.810046,0.0,0.750000,0.750000,0.750000,0.766667,0.750000,0.0,...,419.930636,942.630328,0.001339,0.000336,11.229733,22.489150,4.243500,67.764183,68.643883,0.0
2016-01-31 20:00:00,2016-01-31 20:00:00,1.546231,4.917617,0.0,0.750000,0.750000,0.750000,0.483333,1.000000,0.0,...,419.930636,942.630328,0.001503,0.000385,11.366567,22.483300,4.275667,67.889300,68.772900,0.0
2016-01-31 21:00:00,2016-01-31 21:00:00,1.617565,4.820712,0.0,1.000000,1.000000,1.000000,0.000000,1.000000,0.0,...,419.930636,942.630328,0.001470,0.000437,11.448617,22.509617,4.297500,67.785950,68.651417,0.0


In [46]:
gen_df = year2[year2["PV_Watts3PhTotalW3PhT1"].isna() == False]
gen_df = gen_df[gen_df["PV_Watts3PhTotalW3PhT2"].isna() == False] #There are 87 rows with NaN values.

print(f"A total of {len(year2) - len(gen_df)} rows have been dropped since they have NaN values.")

gen_df["Generated Energy"] = (gen_df["PV_Watts3PhTotalW3PhT1"].to_numpy() + gen_df["PV_Watts3PhTotalW3PhT2"].to_numpy())
gen_df = gen_df[["Generated Energy"]] / 1e3 #Convert Wh to kWh

gen_df = gen_df.resample('h').sum() #Resample to 'h' again to get correct freq + former dropped rows now are back with 0 gen energy
gen_df

A total of 87 rows have been dropped since they have NaN values.


,Generated Energy
Timestamp,
2015-02-01 00:00:00,0.018496
2015-02-01 01:00:00,0.018332
2015-02-01 02:00:00,0.018385
2015-02-01 03:00:00,0.018502
2015-02-01 04:00:00,0.018524
...,...
2016-01-31 19:00:00,0.017710
2016-01-31 20:00:00,0.017940
2016-01-31 21:00:00,0.018149


In [47]:
gen_df_no_changes = gen_df

In [48]:
fig = go.Figure(go.Scattergl(
    x = gen_df.index,
    y = gen_df["Generated Energy"]
))

fig.show()

#### Train, Validation and Test data

In [49]:
X = gen_df.copy()

In [50]:
prediction_range = 24 * 3  #Three days

In [51]:
test_df = X[len(X) - prediction_range : ] #The last 3 days of data
test_df

,Generated Energy
Timestamp,
2016-01-29 00:00:00,0.018250
2016-01-29 01:00:00,0.018189
2016-01-29 02:00:00,0.018158
2016-01-29 03:00:00,0.018049
2016-01-29 04:00:00,0.018243
...,...
2016-01-31 19:00:00,0.017710
2016-01-31 20:00:00,0.017940
2016-01-31 21:00:00,0.018149


In [52]:
X.drop(test_df.index, inplace=True)

validation_df = X[len(X) - prediction_range : ] #The last 3 days of data after dropping the test_df indexes
validation_df

,Generated Energy
Timestamp,
2016-01-26 00:00:00,0.000000
2016-01-26 01:00:00,0.000000
2016-01-26 02:00:00,0.000000
2016-01-26 03:00:00,0.000000
2016-01-26 04:00:00,0.000000
...,...
2016-01-28 19:00:00,0.017729
2016-01-28 20:00:00,0.017849
2016-01-28 21:00:00,0.017882


In [53]:
train_df = X[ : len(X)-prediction_range] #Rest of the data that is not validation or test
train_df_1week = train_df.index[8448] # Used for figures later
train_df_1week, train_df

(Timestamp('2016-01-19 00:00:00', freq='H'),
                      Generated Energy
 Timestamp                            
 2015-02-01 00:00:00          0.018496
 2015-02-01 01:00:00          0.018332
 2015-02-01 02:00:00          0.018385
 2015-02-01 03:00:00          0.018502
 2015-02-01 04:00:00          0.018524
 ...                               ...
 2016-01-25 19:00:00          0.014773
 2016-01-25 20:00:00          0.000000
 2016-01-25 21:00:00          0.000000
 2016-01-25 22:00:00          0.000000
 2016-01-25 23:00:00          0.000000
 
 [8616 rows x 1 columns])

#### Check if PV is stationary + Differencing

In [54]:
from statsmodels.tsa.stattools import adfuller

ADF_test = adfuller(gen_df["Generated Energy"])

print(f"The p-value from the Augmented Dickey-Fuller test is {ADF_test[1]}. \n This suggest that the TS is stationary, and d=0.")

The p-value from the Augmented Dickey-Fuller test is 4.569102377280105e-19. 
 This suggest that the TS is stationary, and d=0.


In [55]:
# Doing the differencing if needed for later

gen_df['1dif'] = gen_df["Generated Energy"] - gen_df["Generated Energy"].shift(1)
gen_df

,Generated Energy,1dif
Timestamp,,
2015-02-01 00:00:00,0.018496,NaN
2015-02-01 01:00:00,0.018332,-0.000165
2015-02-01 02:00:00,0.018385,0.000053
2015-02-01 03:00:00,0.018502,0.000117
2015-02-01 04:00:00,0.018524,0.000022
...,...,...
2016-01-31 19:00:00,0.017710,0.000288
2016-01-31 20:00:00,0.017940,0.000230
2016-01-31 21:00:00,0.018149,0.000208


#### Determine order of autoregressive terms (p) and moving average terms (q)

In [56]:
acf_values, acf_conf = acf(gen_df['Generated Energy'].dropna(), nlags=49, alpha=0.05)

for i in range(len(acf_values)):
    acf_conf[i] = acf_conf[i] - acf_values[i]

acf_conf = pd.DataFrame(acf_conf, columns=['Upper', 'Lower'])

fig = go.Figure(go.Bar(
    y = acf_values
))

fig.add_trace(go.Scatter(
    x = list(acf_conf.index) + list(acf_conf.index[::-1]),
    y = list(acf_conf['Upper']) + list(acf_conf['Lower'][::-1]),
    fill='toself',
    fillcolor='rgba(84,84,84,0.2)',
    line=dict(color='rgba(255,255,255,0)'),
    hoverinfo="skip",
    showlegend=False
))

fig.update_yaxes(title="Autocorrelation", range=[-1, 1])
fig.update_xaxes(title="Lag")

fig.show()

#autocorrelation_no_difference = plot_acf(gen_df["Generated Energy"].dropna())
# autocorrelation_first_difference = plot_acf(gen_df["1dif"].dropna())

In [57]:
pacf_values, pacf_conf = pacf(gen_df['Generated Energy'].dropna(), nlags=49, alpha=0.05)

for i in range(len(pacf_values)):
    pacf_conf[i] = pacf_conf[i] - pacf_values[i]

pacf_conf = pd.DataFrame(pacf_conf, columns=['Upper', 'Lower'])

fig = go.Figure(go.Bar(
    y = pacf_values
))

fig.add_trace(go.Scatter(
    x = list(pacf_conf.index) + list(pacf_conf.index[::-1]),
    y = list(acf_conf['Upper']) + list(acf_conf['Lower'][::-1]),
    fill='toself',
    fillcolor='rgba(84,84,84,0.2)',
    line=dict(color='rgba(255,255,255,0)'),
    hoverinfo="skip",
    showlegend=False
))

fig.update_yaxes(title="Partial Autocorrelation", range=[-1, 1])
fig.update_xaxes(title="Lag")

fig.show()

#partialautocorrelation_no_difference = plot_pacf(gen_df['Generated Energy'].dropna())
#partialautocorrelation_first_difference = plot_pacf(gen_df['1dif'].dropna())

#### ARIMA

In [58]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

def arimamodel(*, training_data, trend='c', p, d, q, P=0, D=0, Q=0, m=0,):

    model = SARIMAX(training_data, trend=trend, order=(p, d, q), seasonal_order=(P, D, Q, m))
    fitted_model = model.fit()

    return fitted_model

def arima_forecast(data, fitted_model, *, start_index, end_index):

    data['Prediction'] = fitted_model.predict(start = start_index, end = end_index)

    predictions = fitted_model.get_forecast(end_index)
    #yhat = test.predicted_mean
    yhat_conf = predictions.conf_int(alpha=0.9)

    return data, yhat_conf


def plot_forecast(data, conf_interval, bands=True):

    fig = go.Figure(go.Scattergl(
        x = data.index,
        y = data['Generated Energy'],
        name = "Observed Values"
    ))

    fig.add_trace(go.Scattergl(
        x = data.index,
        y = data['Prediction'],
        name = "Predicted Values"
    ))

    if bands == True:
        fig.add_trace(go.Scattergl(
            x = conf_interval.index,
            y = conf_interval['lower Generated Energy'],
            name = "Lower Bound"
        ))

        fig.add_trace(go.Scattergl(
            x = conf_interval.index,
            y = conf_interval['upper Generated Energy'],
            name = "Upper Bound"
        ))

    fig.show()

In [59]:
model1 = arimamodel(training_data=train_df['Generated Energy'], trend='c', p=4, d=0, q=0)
model1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:       Generated Energy   No. Observations:                 8616
Model:               SARIMAX(4, 0, 0)   Log Likelihood              -10385.704
Date:                Sun, 08 May 2022   AIC                          20783.407
Time:                        19:40:39   BIC                          20825.776
Sample:                    02-01-2015   HQIC                         20797.856
                         - 01-25-2016                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.2671      0.019     13.932      0.000       0.230       0.305
ar.L1          1.2682      0.007    176.425      0.000       1.254       1.282
ar.L2         -0.2803      0.009    -32.235      0.000      -0.297      -0.263
ar.L3         -0.0041      0.011     -0.382      0.703      -0.025       0.017
ar.L4         -0.1514      0.008    -19.117      0.000      -0.167      -0.136
sigma2         0.6522      0.006    112.030      0.000       0.641       0.664
===================================================================================
Ljung-Box (L1) (Q):                   0.03   Jarque-Bera (JB):             34883.09
Prob(Q):                              0.86   Prob(JB):                         0.00
Heteroskedasticity (H):               0.68   Skew:                             0.79
Prob(H) (two-sided):                  0.00   Kurtosis:                        12.73
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [60]:
forecast1, conf_int1 = arima_forecast(data=gen_df, fitted_model=model1, start_index=0, end_index=gen_df.index[-1])
forecast1

,Generated Energy,1dif,Prediction
Timestamp,,,
2015-02-01 00:00:00,0.018496,NaN,1.593420
2015-02-01 01:00:00,0.018332,-0.000165,0.143705
2015-02-01 02:00:00,0.018385,0.000053,0.209233
2015-02-01 03:00:00,0.018502,0.000117,0.247685
2015-02-01 04:00:00,0.018524,0.000022,0.282544
...,...,...,...
2016-01-31 19:00:00,0.017710,0.000288,1.593420
2016-01-31 20:00:00,0.017940,0.000230,1.593420
2016-01-31 21:00:00,0.018149,0.000208,1.593420


In [61]:
from sklearn.metrics import mean_squared_error

y_true = forecast1['Generated Energy'][validation_df.index[0] : validation_df.index[-1]]
y_pred = forecast1['Prediction'][validation_df.index[0] : validation_df.index[-1]]

model1_rmse = mean_squared_error(y_true,y_pred, squared=False) #squared=False returns RMSE, True returns MSE
model1_rmse

2.2988386673167733

In [62]:
fig = go.Figure(go.Scattergl(
    x = forecast1[train_df_1week : validation_df.index[0]].index,
    y = forecast1['Generated Energy'][train_df_1week : validation_df.index[0]],
    name = "Training Data",
    line=dict(color='rgb(84, 84, 84)')
))

fig.add_trace(go.Scattergl(
    x = forecast1[validation_df.index[0] : validation_df.index[-1]].index,
    y = forecast1['Generated Energy'][validation_df.index[0] : validation_df.index[-1]],
    name = "Observed",
    line=dict(color='rgb(234,143,129)')
))

fig.add_trace(go.Scattergl(
    x = forecast1[validation_df.index[0]:validation_df.index[-1]].index,
    y = forecast1['Prediction'][validation_df.index[0]:validation_df.index[-1]],
    name = "Predicted",
    line=dict(color='rgb(32,115,171)')
))

# if bands == True:
#     fig.add_trace(go.Scattergl(
#         x = conf_interval.index,
#         y = conf_interval['lower Generated Energy'],
#         name = "Lower Bound"
#     ))
#
#     fig.add_trace(go.Scattergl(
#         x = conf_interval.index,
#         y = conf_interval['upper Generated Energy'],
#         name = "Upper Bound"
#     ))

fig.update_yaxes(title="Generated Energy [kWh]")

fig.show()

## Seasonal ARIMA

#### Calculate seasonal difference

In [63]:
# Calculate the seasonal difference as a daily seasonality.
gen_df["Seasonal Difference"] = gen_df["Generated Energy"] - gen_df["Generated Energy"].shift(24)
gen_df

,Generated Energy,1dif,Prediction,Seasonal Difference
Timestamp,,,,
2015-02-01 00:00:00,0.018496,NaN,1.593420,NaN
2015-02-01 01:00:00,0.018332,-0.000165,0.143705,NaN
2015-02-01 02:00:00,0.018385,0.000053,0.209233,NaN
2015-02-01 03:00:00,0.018502,0.000117,0.247685,NaN
2015-02-01 04:00:00,0.018524,0.000022,0.282544,NaN
...,...,...,...,...
2016-01-31 19:00:00,0.017710,0.000288,1.593420,0.000077
2016-01-31 20:00:00,0.017940,0.000230,1.593420,0.000123
2016-01-31 21:00:00,0.018149,0.000208,1.593420,0.000199


In [64]:
test = adfuller(gen_df["Seasonal Difference"].dropna())

print(f"The p value for the seasonal difference is {test[1]}. \nThis suggest D=0.")

The p value for the seasonal difference is 0.0. 
This suggest D=0.


In [65]:
fig = go.Figure(go.Scatter(
    x = gen_df.index,
    y = gen_df["Seasonal Difference"]
))

fig.update_yaxes(title="Generated Energy [kWh]")

fig.show()

### SAR and SMA terms

In [66]:
seasonal_acf_values, seasonal_acf_conf = acf(gen_df['Seasonal Difference'].dropna(), nlags=49, alpha=0.05)
seasonal_pacf_values, seasonal_pacf_conf = pacf(gen_df['Seasonal Difference'].dropna(), nlags=49, alpha=0.05)

for i in range(len(seasonal_acf_values)):
    seasonal_acf_conf[i] = seasonal_acf_conf[i] - seasonal_acf_values[i]

seasonal_acf_conf = pd.DataFrame(seasonal_acf_conf, columns=['Upper', 'Lower'])

for i in range(len(seasonal_pacf_values)):
    seasonal_pacf_conf[i] = seasonal_pacf_conf[i] - seasonal_pacf_values[i]

seasonal_pacf_conf = pd.DataFrame(seasonal_pacf_conf, columns=['Upper', 'Lower'])

In [67]:
fig = go.Figure(go.Bar(
    y = seasonal_acf_values
))

fig.add_trace(go.Scatter(
    x = list(seasonal_acf_conf.index) + list(seasonal_acf_conf.index[::-1]),
    y = list(seasonal_acf_conf['Upper']) + list(seasonal_acf_conf['Lower'][::-1]),
    fill='toself',
    fillcolor='rgba(84,84,84,0.2)',
    line=dict(color='rgba(255,255,255,0)'),
    hoverinfo="skip",
    showlegend=False
))

fig.update_yaxes(title="Autocorrelation", range=[-1, 1])
fig.update_xaxes(title="Lag")

fig.show()

In [68]:
fig = go.Figure(go.Bar(
    y = seasonal_pacf_values
))

fig.add_trace(go.Scatter(
    x = list(seasonal_pacf_conf.index) + list(seasonal_pacf_conf.index[::-1]),
    y = list(seasonal_acf_conf['Upper']) + list(seasonal_acf_conf['Lower'][::-1]),
    fill='toself',
    fillcolor='rgba(84,84,84,0.2)',
    line=dict(color='rgba(255,255,255,0)'),
    hoverinfo="skip",
    showlegend=False
))

fig.update_yaxes(title="Partial Autocorrelation", range=[-1, 1])
fig.update_xaxes(title="Lag")

fig.show()

## SARIMA(4, 0, 0, 1, 0, 0, 24) + SARIMA(4, 0, 0, 0, 1, 2, 24)

In [ ]:
model2 = arimamodel(training_data=train_df['Generated Energy'], trend='c', p=4, d=0, q=0, P=0, D=1, Q=2, m=24)
print(model2.summary())

In [ ]:
forecast2, conf_int2 = arima_forecast(data=gen_df, fitted_model=model2, start_index=0, end_index=gen_df.index[-1])
forecast2

In [ ]:
y_true = forecast2['Generated Energy'][validation_df.index[0] : validation_df.index[-1]]
y_pred = forecast2['Prediction'][validation_df.index[0] : validation_df.index[-1]]

model2_rmse = mean_squared_error(y_true,y_pred, squared=False) #squared=False returns RMSE, True returns MSE
model2_rmse

In [ ]:
fig = go.Figure(go.Scattergl(
    x = forecast2[train_df_1week : validation_df.index[0]].index,
    y = forecast2['Generated Energy'][train_df_1week : validation_df.index[0]],
    name = "Training Data",
    line=dict(color='rgb(84, 84, 84)')
))

fig.add_trace(go.Scattergl(
    x = forecast2[validation_df.index[0] : validation_df.index[-1]].index,
    y = forecast2['Generated Energy'][validation_df.index[0] : validation_df.index[-1]],
    name = "Observed",
    line=dict(color='rgb(234,143,129)')
))

fig.add_trace(go.Scattergl(
    x = forecast2[validation_df.index[0]:validation_df.index[-1]].index,
    y = forecast2['Prediction'][validation_df.index[0]:validation_df.index[-1]],
    name = "Predicted",
    line=dict(color='rgb(32,115,171)')
))

# if bands == True:
#     fig.add_trace(go.Scattergl(
#         x = conf_int2.index,
#         y = conf_int2['lower Generated Energy'],
#         name = "Lower Bound"
#     ))
#
#     fig.add_trace(go.Scattergl(
#         x = conf_int2.index,
#         y = conf_int2['upper Generated Energy'],
#         name = "Upper Bound"
#     ))

fig.update_yaxes(title="Generated Energy [kWh]")

fig.show()

#### Testing on test set

In [ ]:
forecast_test, conf_int_test = arima_forecast(data=gen_df, fitted_model=model2, start_index=0, end_index=gen_df.index[-1])
forecast_test

In [ ]:
y_true = forecast_test['Fixed Consumed'][test_df.index[0] : test_df.index[-1]]
y_pred = forecast_test['Prediction'][test_df.index[0] : test_df.index[-1]]

model2_test_rmse = mean_squared_error(y_true,y_pred, squared=False) #squared=False returns RMSE, True returns MSE
model2_test_rmse

In [ ]:
fig = go.Figure(go.Scattergl(
    x = forecast_test[train_df_1week : test_df.index[0]].index,
    y = forecast_test['Fixed Consumed'][train_df_1week : test_df.index[0]],
    line=dict(color='rgb(84, 84, 84)'),
    name = "Training Data"
))

fig.add_trace(go.Scattergl(
    x = forecast_test[test_df.index[0] : test_df.index[-1]].index,
    y = forecast_test['Fixed Consumed'][test_df.index[0] : test_df.index[-1]],
    line=dict(color='rgb(234,143,129)'),
    name = "Test Data"
))

fig.add_trace(go.Scattergl(
    x = forecast_test[test_df.index[0] : test_df.index[-1]].index,
    y = forecast_test['Prediction'][test_df.index[0] : test_df.index[-1]],
    name = "Predicted",
    line=dict(color='rgb(32,115,171)')
))

fig.update_yaxes(title="Fixed Consumed Energy [kWh]")

fig.show()

### SARIMAX(?, ?, ?, ?, ?, ?, 24)

In [ ]:
from sklearn.preprocessing import OneHotEncoder

gen_df["Hour"] = gen_df.index.hour

ohe = OneHotEncoder(sparse=False)
hot_np = ohe.fit_transform(gen_df[["Hour"]])
hot = pd.DataFrame(data=hot_np, columns=ohe.get_feature_names_out())
hot

#exog_attributes = pd.DataFrame(year2['PV_PVInsolationHArray'].resample('h').sum()) # If we want exogenous attributes in the model as well.

# X = pd.merge(gen_df, exog_attributes, how='left', on=gen_df.index)
# X.drop(columns=['key_0'], inplace=True)
# X.index = gen_df.index
#X.drop(columns=['1dif', '2dif', '3dif', 'Seasonal Difference'], inplace=True)

In [ ]:
gen_df_no_changes =  gen_df_no_changes.reset_index()
gen_df_no_changes.drop(columns=['1dif', 'Prediction', 'Seasonal Difference', 'Hour'], inplace=True)
gen_df_no_changes

In [ ]:
X = gen_df_no_changes.join(hot)
X

In [ ]:
weather_forecast = pd.read_csv("C://Users//magnu//OneDrive - Aalborg Universitet//P6//Data//weather_data//GAI_2015_2016.csv")
weather_forecast.rename(columns={'Time': 'Timestamp'}, inplace=True)
weather_forecast['Timestamp'] = pd.to_datetime(weather_forecast['Timestamp'])
weather_forecast

In [ ]:
time_range = pd.DataFrame(pd.date_range(start=weather_forecast['Timestamp'][0], end=weather_forecast['Timestamp'][len(weather_forecast)-1], freq='H'), columns=['Timestamp'])
time_range

In [ ]:
weather_forecast = time_range.merge(weather_forecast, on='Timestamp', how='left') # We need to remake the weather_forecast dataframe with the new time_range to keep the forward filled NaN values for later. Else they will be 0-0-0-0
weather_forecast.ffill(axis='rows', inplace=True) # There are 42 conditions with NaN values so we fill from previous observations.
weather_forecast

In [ ]:
ohe_condition = OneHotEncoder(sparse=False)
hot_condition = ohe_condition.fit_transform(weather_forecast[["Condition"]])
conditions = pd.DataFrame(data=hot_condition, columns=ohe_condition.get_feature_names_out())
conditions

In [ ]:
X = X.merge(conditions, left_index=True, right_index=True, how="left")
X

In [ ]:
exog_attributes = list(X.columns)
exog_attributes.remove('Timestamp')
exog_attributes.remove('Generated Energy')
exog_attributes

In [ ]:
# model = SARIMAX(X['Generated Energy'], exog=X[exog_attributes], trend='c',
#                  order=(3, 0, 2), seasonal_order=(0, 1, 2, 24))
# model3 = model.fit()
# print(model3.summary())

In [ ]:
forecast3, conf_int3 = arima_forecast(data=gen_df, fitted_model=model3, start_index=0, end_index=validation_df.index[-1])
forecast3

In [ ]:
y_true = forecast3['Generated Energy'][validation_df.index[0] : validation_df.index[-1]]
y_pred = forecast3['Prediction'][validation_df.index[0] : validation_df.index[-1]]

model3_rmse = mean_squared_error(y_true,y_pred, squared=False) #squared=False returns RMSE, True returns MSE
model3_rmse

In [ ]:
fig = go.Figure(go.Scattergl(
    x = forecast3[train_df_1week : validation_df.index[0]].index,
    y = forecast3['Generated Energy'][train_df_1week : validation_df.index[0]],
    name = "Training Data",
    line=dict(color='rgb(84, 84, 84)')
))

fig.add_trace(go.Scattergl(
    x = forecast3[validation_df.index[0] : validation_df.index[-1]].index,
    y = forecast3['Generated Energy'][validation_df.index[0] : validation_df.index[-1]],
    name = "Observed",
    line=dict(color='rgb(234,143,129)')
))

fig.add_trace(go.Scattergl(
    x = forecast3[validation_df.index[0]:validation_df.index[-1]].index,
    y = forecast3['Prediction'][validation_df.index[0]:validation_df.index[-1]],
    name = "Predicted",
    line=dict(color='rgb(32,115,171)')
))

# if bands == True:
#     fig.add_trace(go.Scattergl(
#         x = conf_int2.index,
#         y = conf_int2['lower Generated Energy'],
#         name = "Lower Bound"
#     ))
#
#     fig.add_trace(go.Scattergl(
#         x = conf_int2.index,
#         y = conf_int2['upper Generated Energy'],
#         name = "Upper Bound"
#     ))

fig.update_yaxes(title="Generated Energy [kWh]")

fig.show()

### Future values

In [ ]:
# If we need to predict future values

# from pandas.tseries.offsets import DateOffset
#
# new_dates = [gen_df.index[-1]+DateOffset(hours=x) for x in range(1, 26) ]
# new_dates

# gen_df_predict = pd.DataFrame(index=new_dates, columns=gen_df.columns)
# test_df = pd.concat([gen_df, gen_df_predict])
# test_df['Prediction'] = fitted_model.predict(start=8759, end=8785)
# test_df